In [ ]:
import pandas as pd
import numpy as np
import pymysql
import sys

import utility
from datetime import date

ModuleNotFoundError: No module named 'utility'

# transacting users

In [2]:
transacted_user_q = """
Select user_id, min(created_time) first_transaction, max(created_time) last_transaction, date_format(min(created_time), '%Y-%m') as first_transaction_month
from merchant_payments
where status='success'
group by user_id
"""
parse_date = {'first_transaction' : {"format" : '%Y-%m-%d %H:%M:%S'},
              'last_transaction' : {"format" : '%Y-%m-%d %H:%M:%S'}}
flag, transacted_user,error_msg = utility.data_fetch_v2(transacted_user_q)

In [3]:
transacted_user.shape

(17465, 4)

# Sanction data

In [4]:
sanction_data = f"""
Select * from sanction
where user_id in {tuple(transacted_user.user_id)}
"""

parse_date = {'first_transaction' : {"format" : '%Y-%m-%d %H:%M:%S'},
              'last_transaction' : {"format" : '%Y-%m-%d %H:%M:%S'}}
flag, sanction_data,error_msg = utility.data_fetch_v2(sanction_data)
users_4m = sanction_data[sanction_data['prod_group']=='4M']
users_4m.shape

(11674, 19)

# marchent paymant

In [5]:
merchant_payments = f"""
SELECT mp.user_id,
date_format(mp.created_time, '%Y%m') billing_cycle,
SUM(amount) AS disbursal
FROM merchant_payments AS mp
WHERE mp.`status` = 'success'
GROUP BY 1,2
"""
parse_date = {}
# otp_varified_user=pd.read_sql(otp_varified, db_connection)
flag, merchant_payments_data, error = utility.data_fetch_v2(merchant_payments, parse_dates=parse_date)
merchant_payments_data.shape

(94358, 3)

In [6]:
merchant_payments_data

,user_id,billing_cycle,disbursal
0,000ade4b-897e-450a-89f2-2f5e3078737e,202304,4832.0
1,000ade4b-897e-450a-89f2-2f5e3078737e,202305,4778.0
2,000ade4b-897e-450a-89f2-2f5e3078737e,202306,4799.0
3,000ade4b-897e-450a-89f2-2f5e3078737e,202307,4800.0
4,000ade4b-897e-450a-89f2-2f5e3078737e,202308,1186.0
...,...,...,...
94353,fffe2a21-2db8-4402-b21e-4bf824900493,202302,5000.0
94354,fffe2a21-2db8-4402-b21e-4bf824900493,202303,10001.0
94355,fffe2a21-2db8-4402-b21e-4bf824900493,202304,10000.0
94356,fffe2a21-2db8-4402-b21e-4bf824900493,202305,7000.0


In [7]:
merchant_payments_data[(merchant_payments_data['user_id'].isin(users_4m.user_id))]['disbursal'].sum(), len(set(merchant_payments_data.user_id))

(518954597.56999993, 17465)

# Billing info

In [8]:
billing_info_q = """
Select * 
from billing_info
"""
flag, billing_info,error_msg = utility.data_fetch_v2(billing_info_q)

In [36]:
DPD = billing_info[(billing_info['billing_month']=='December')&(billing_info['billing_year']=='2024')][['user_id', 'DPD','principal_outstanding','payment_max', "billing_cycle"]]
DPD[DPD['user_id'].isin(organic_users_table[organic_users_table['new_policy_approved']==1]['user_id'])]['principal_outstanding'].sum()

12425253.56

In [47]:
DPD

,user_id,DPD,principal_outstanding,payment_max,billing_cycle
301385,1f2b7fe3-77e6-43e3-96aa-7cf85f7f838e,1484,6000.0,8194.8,202412
301386,d0c030ac-8e0a-42d0-8d8e-9fd22f95b5ae,1484,4476.0,6387.6,202412
301387,b77738e1-4139-46c8-a549-7277f16c6d76,358,3720.5,5313.5,202412
301388,4b08627d-31f1-49c4-85ae-0cd3e8d1f8f9,937,4141.6,5770.0,202412
301389,d3f2368f-6e3f-4f06-82d0-019f4f792bde,0,0.0,0.0,202412
...,...,...,...,...,...
306598,aa3965c3-b18f-4325-ac68-19446f04d335,0,0.0,0.0,202412
306599,b7b3f6b9-0f26-4bd5-b777-c3289f955034,389,4000.0,5062.0,202412
306600,4228b2dd-36a3-4a72-8eb2-fe3c29b61243,389,1000.0,2062.0,202412
306601,c03f7619-3e31-413b-8b0d-5f7b8ca106d9,389,3800.0,4862.0,202412


# model score

## sms

In [9]:
sms_model_score = f"""
select * from asp_playground.sms_gbm_model_score_for_transacting_user
where user_id in {tuple(transacted_user.user_id)}
"""
flag, sms_model_score_data,error_msg = utility.data_fetch_v2(sms_model_score)

# model_score_not_present_users= transacted_user[~transacted_user['user_id'].isin(sms_model_score_data.user_id)]

sms_data = pd.read_csv(r'C:\Users\Niveditha\Desktop\Data_Science\aspire\sms pipeline\SMS Features\sms_file_output\sms_score_output_pred_recently_trns.csv')
sms_data = sms_data.rename(columns = {"prediction":"sms_gbm_risk_model_score"})
sms_data.shape

sms_model_score_ = sms_model_score_data[['user_id', 'sms_gbm_risk_model_score']].append(sms_data[['user_id', 'sms_gbm_risk_model_score']])
sms_model_score_.shape, len(set(sms_model_score_.user_id))

((17195, 2), 17195)

## bureau

In [10]:
bureau_model_score = f"""
select * from asp_playground.crif_risk_model_v2_score_s10_upto_nov282023
where user_id in {tuple(users_4m.user_id)}
"""
flag, bureau_model_score_data,error_msg = utility.data_fetch_v2(bureau_model_score)
bureau_model_score_data.columns
bureau_model_score_data = bureau_model_score_data.drop_duplicates('user_id', keep='last')

In [11]:
transacted_user =transacted_user.merge(sms_model_score_, on = 'user_id', how = 'left')
transacted_user =transacted_user.merge(bureau_model_score_data, on = 'user_id', how = 'left')

In [ ]:


statement_history = f"""
select billing_cycle, count(distinct user_id) from statement_history
group by billing_cycle
"""
flag, statement_history_,error_msg = utility.data_fetch_v2(statement_history)
statement_history_

# Organic

In [12]:
def policy_organic(df):
    approve = []
    for i in range(len(df)):

        if ((df['crif_lr_risk_model_score'][i]>0)&(df['crif_lr_risk_model_score'][i]<0.89)&
            (df['sms_gbm_risk_model_score'][i]>0)&(df['sms_gbm_risk_model_score'][i]<0.3499)):
            approve.append(1)
        elif ((df['crif_lr_risk_model_score'][i]>0)&(df['crif_lr_risk_model_score'][i]<=0.31)&
            (df['sms_gbm_risk_model_score'][i]>=0.3499)&(df['sms_gbm_risk_model_score'][i]<0.7725)):
            approve.append(1)
        elif (np.isnan(df['crif_lr_risk_model_score'][i])):
            approve.append(1)
        else:
            approve.append(0)
    df['new_policy_approved'] = approve
    return df

In [13]:
organic_users = sanction_data[(sanction_data['prod_group']=='4M')]
organic_users_table =transacted_user[transacted_user['user_id'].isin(organic_users.user_id)][['user_id','crif_lr_risk_model_score','sms_gbm_risk_model_score']].reset_index(drop =True)
organic_users_table = policy_organic(organic_users_table)

In [14]:
organic_users_table['new_policy_approved'].value_counts()


1    8007
0    3667
Name: new_policy_approved, dtype: int64

In [15]:
users_4m = users_4m[users_4m['user_id'].isin(organic_users_table[organic_users_table['new_policy_approved']==1]['user_id'])].reset_index(drop = True)
users_4m.shape

(8007, 19)

# fee paid 4M

In [16]:
'''
fee cohort
'''

users_4m = sanction_data[sanction_data['prod_group']=='4M']
# users_4m = users_4m[users_4m['user_id'].isin(organic_users_table[organic_users_table['new_policy_approved']==1]['user_id'])].reset_index(drop = True)


def statement_h_fun(users_not_there,billing_cycle):
    fee_paid  = f"""
    SELECT s.user_id,
    (case when MONTH(s.due_date)<10 then concat(YEAR(s.due_date),"0", MONTH(s.due_date)) ELSE concat(YEAR(s.due_date),MONTH(s.due_date))  END) AS billing_cycle,
    SUM((case when s.payment_status = "Paid" AND s.charge_type = "bMonthly"  then s.amount/1.18 ELSE 0 END)) AS monthly_fee_paid,
    SUM((case when s.payment_status = "Paid" AND s.charge_type = "cLatefee"  then s.amount/1.18 ELSE 0 END)) AS late_fee_paid
    FROM statement_history AS s
    WHERE s.charge_type <> 'aTranch' and user_id in {tuple(users_not_there.user_id)}
    and billing_cycle = {billing_cycle}
    GROUP BY 1,2
    """
    parse_date = {}
    # otp_varified_user=pd.read_sql(otp_varified, db_connection)
    flag,fee_paid_data1, error_msg = utility.data_fetch_v2(fee_paid)
    # fee_paid_data.head()
    return fee_paid_data1

In [27]:
fee_paid_data = pd.DataFrame({'user_id' : []})
billing_cycle_list = ['202407','202406','202405','202404','202403','202402','202401','202311','202310','202309','202308','202307','202306','202305',
                      '202304','202303','202302','202301','202212','202211',
                      '202210','202209','202208','202207','202206','202205','202204',
                      '202203','202202','202201','202112', '202111','202110','202109']
for i in range(len(billing_cycle_list)): 
    users_not_there = users_4m[~(users_4m['user_id'].isin(fee_paid_data.user_id))]
    print(len(users_not_there))
    fee_paid_chunk = statement_h_fun(users_not_there,billing_cycle_list[i])
    fee_paid_data = fee_paid_data.append(fee_paid_chunk)

11674
7551
7547
7516
7467
7424
6921
1187
1082
1047
1007
920
865
763
726
410
385
370
345
327
296
272
252
166
166
166
166
166
166
166
166
166
166
166


## Merging

In [28]:
fee_paid_data.shape, organic_users_table.shape, len(set(organic_users_table.user_id))

((128053, 4), (11674, 4), 11674)

In [29]:
fee_paid_data = fee_paid_data.merge(organic_users_table[["user_id","new_policy_approved"]],  on ="user_id", how = "left")

In [30]:
# fee_paid_data[["new_policy_approved","monthly_fee_paid"]].value_counts()
fee_paid_data

,user_id,billing_cycle,monthly_fee_paid,late_fee_paid,new_policy_approved
0,000ade4b-897e-450a-89f2-2f5e3078737e,202305,214.79661,0.0,0
1,000ade4b-897e-450a-89f2-2f5e3078737e,202306,214.79661,0.0,0
2,000ade4b-897e-450a-89f2-2f5e3078737e,202307,214.79661,0.0,0
3,000ade4b-897e-450a-89f2-2f5e3078737e,202308,214.79661,0.0,0
4,000ade4b-897e-450a-89f2-2f5e3078737e,202309,0.00000,0.0,0
...,...,...,...,...,...
128048,fd7ec252-76d6-43ac-8532-cf9bbe1ea24e,202208,0.00000,300.0,0
128049,ffe398bb-08eb-4570-be61-fa9792534075,202202,90.00000,0.0,1
128050,ffe398bb-08eb-4570-be61-fa9792534075,202205,90.00000,0.0,1
128051,ffe398bb-08eb-4570-be61-fa9792534075,202206,90.00000,0.0,1


In [31]:
fee_paid_data.groupby("new_policy_approved")["monthly_fee_paid"].sum().round(0)
# fee_paid_data.columns

new_policy_approved
0     6855996.0
1    16795350.0
Name: monthly_fee_paid, dtype: float64

In [33]:
fee_paid_data.groupby("new_policy_approved")["late_fee_paid"].sum().round(0)


new_policy_approved
0     808910.0
1    1205403.0
Name: late_fee_paid, dtype: float64

In [25]:
fee_paid_data

,user_id,billing_cycle,monthly_fee_paid,late_fee_paid,new_policy_approved
0,000ade4b-897e-450a-89f2-2f5e3078737e,202305,429.59322,0.0,0
1,000ade4b-897e-450a-89f2-2f5e3078737e,202306,429.59322,0.0,0
2,000ade4b-897e-450a-89f2-2f5e3078737e,202307,429.59322,0.0,0
3,000ade4b-897e-450a-89f2-2f5e3078737e,202308,429.59322,0.0,0
4,000ade4b-897e-450a-89f2-2f5e3078737e,202309,0.00000,0.0,0
...,...,...,...,...,...
128048,fd7ec252-76d6-43ac-8532-cf9bbe1ea24e,202208,0.00000,300.0,0
128049,ffe398bb-08eb-4570-be61-fa9792534075,202202,90.00000,0.0,1
128050,ffe398bb-08eb-4570-be61-fa9792534075,202205,90.00000,0.0,1
128051,ffe398bb-08eb-4570-be61-fa9792534075,202206,90.00000,0.0,1


In [49]:
fee_paid_data.columns

Index(['user_id', 'billing_cycle', 'monthly_fee_paid', 'late_fee_paid',
       'new_policy_approved'],
      dtype='object')

In [25]:
# organic_users_table = organic_users_table[organic_users_table['new_policy_approved']==1].reset_index(drop = True)

In [ ]:
tol_disbursed_amt = merchant_payments_data.groupby('user_id')['disbursal'].sum().rename('total_disbursed').reset_index()
final = organic_users_table.merge(tol_disbursed_amt, on = 'user_id', how= 'left')
final = final.merge(DPD, on = 'user_id', how= 'left')
final

In [51]:
DPD

,user_id,DPD,principal_outstanding,payment_max,billing_cycle
301385,1f2b7fe3-77e6-43e3-96aa-7cf85f7f838e,1484,6000.0,8194.8,202412
301386,d0c030ac-8e0a-42d0-8d8e-9fd22f95b5ae,1484,4476.0,6387.6,202412
301387,b77738e1-4139-46c8-a549-7277f16c6d76,358,3720.5,5313.5,202412
301388,4b08627d-31f1-49c4-85ae-0cd3e8d1f8f9,937,4141.6,5770.0,202412
301389,d3f2368f-6e3f-4f06-82d0-019f4f792bde,0,0.0,0.0,202412
...,...,...,...,...,...
306598,aa3965c3-b18f-4325-ac68-19446f04d335,0,0.0,0.0,202412
306599,b7b3f6b9-0f26-4bd5-b777-c3289f955034,389,4000.0,5062.0,202412
306600,4228b2dd-36a3-4a72-8eb2-fe3c29b61243,389,1000.0,2062.0,202412
306601,c03f7619-3e31-413b-8b0d-5f7b8ca106d9,389,3800.0,4862.0,202412


In [38]:
tol_disbursed_amt = merchant_payments_data.groupby('user_id')['disbursal'].sum().rename('total_disbursed').reset_index()
final = organic_users_table.merge(tol_disbursed_amt, on = 'user_id', how= 'left')
final = final.merge(DPD, on = 'user_id', how= 'left')
final = final.merge(fee_paid_data[['user_id', 'billing_cycle', 'monthly_fee_paid', 'late_fee_paid']], on = 'user_id', how= 'left')
# final = final.merge(organic_users_table[['user_id','new_policy_approved']])

merchant_payments_data[merchant_payments_data['user_id'].isin(final.user_id)].to_csv('4m_disbursal_with_date.csv')

In [39]:
fee_paid_data[fee_paid_data['user_id'].isin(final[final["new_policy_approved"]==0]['user_id'])]

,user_id,billing_cycle,monthly_fee_paid,late_fee_paid,new_policy_approved
0,000ade4b-897e-450a-89f2-2f5e3078737e,202305,214.79661,0.0,0
1,000ade4b-897e-450a-89f2-2f5e3078737e,202306,214.79661,0.0,0
2,000ade4b-897e-450a-89f2-2f5e3078737e,202307,214.79661,0.0,0
3,000ade4b-897e-450a-89f2-2f5e3078737e,202308,214.79661,0.0,0
4,000ade4b-897e-450a-89f2-2f5e3078737e,202309,0.00000,0.0,0
...,...,...,...,...,...
128044,fd7ec252-76d6-43ac-8532-cf9bbe1ea24e,202204,15.00000,300.0,0
128045,fd7ec252-76d6-43ac-8532-cf9bbe1ea24e,202205,15.00000,300.0,0
128046,fd7ec252-76d6-43ac-8532-cf9bbe1ea24e,202206,15.00000,300.0,0
128047,fd7ec252-76d6-43ac-8532-cf9bbe1ea24e,202207,0.00000,300.0,0


In [40]:
final.groupby("new_policy_approved")["late_fee_paid"].sum().round(0)

new_policy_approved
0     808910.0
1    1205403.0
Name: late_fee_paid, dtype: float64

In [41]:
len(set(final.user_id))

11674

In [42]:
final.groupby("new_policy_approved")["user_id"].count()

new_policy_approved
0    48451
1    79768
Name: user_id, dtype: int64

In [43]:
final

,user_id,crif_lr_risk_model_score,sms_gbm_risk_model_score,new_policy_approved,total_disbursed,DPD,principal_outstanding,payment_max,billing_cycle_x,billing_cycle_y,monthly_fee_paid,late_fee_paid
0,000ade4b-897e-450a-89f2-2f5e3078737e,0.45,0.3914,0,20395.0,480.0,6239.46,8961.37,202412,202305,214.79661,0.0
1,000ade4b-897e-450a-89f2-2f5e3078737e,0.45,0.3914,0,20395.0,480.0,6239.46,8961.37,202412,202306,214.79661,0.0
2,000ade4b-897e-450a-89f2-2f5e3078737e,0.45,0.3914,0,20395.0,480.0,6239.46,8961.37,202412,202307,214.79661,0.0
3,000ade4b-897e-450a-89f2-2f5e3078737e,0.45,0.3914,0,20395.0,480.0,6239.46,8961.37,202412,202308,214.79661,0.0
4,000ade4b-897e-450a-89f2-2f5e3078737e,0.45,0.3914,0,20395.0,480.0,6239.46,8961.37,202412,202309,0.00000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
128214,fffefa7e-31b7-4d2c-abaa-50304ed774d5,0.51,0.5971,0,4000.0,1029.0,4000.00,6690.40,202412,202301,0.00000,0.0
128215,fffefa7e-31b7-4d2c-abaa-50304ed774d5,0.51,0.5971,0,4000.0,1029.0,4000.00,6690.40,202412,202302,0.00000,0.0
128216,fffefa7e-31b7-4d2c-abaa-50304ed774d5,0.51,0.5971,0,4000.0,1029.0,4000.00,6690.40,202412,202303,0.00000,0.0
128217,fffefa7e-31b7-4d2c-abaa-50304ed774d5,0.51,0.5971,0,4000.0,1029.0,4000.00,6690.40,202412,202304,0.00000,0.0


In [55]:
fee_paid_data_ = billing_info[['user_id','DPD','principal_outstanding','payment_max', 'billing_cycle']].merge(fee_paid_data, on = [ 'user_id','billing_cycle'], how = 'left')

In [60]:
fee_paid_data_[(fee_paid_data_['payment_max']==0)&(fee_paid_data_['user_id'].isin(users_4m.user_id))
               &(fee_paid_data_['monthly_fee_paid']>0)]

,user_id,DPD,principal_outstanding,payment_max,billing_cycle,monthly_fee_paid,late_fee_paid,new_policy_approved
251,b6795f76-dd1c-4cb1-8aaf-2f73d4e12017,0,0.0,0.0,202105,240.000000,0.0,1.0
305,c1772ed2-4a72-4a87-a0e5-31e8fdc9d458,0,0.0,0.0,202105,240.000000,0.0,1.0
345,1ba12d1b-949d-4af0-ae85-ac384dda4651,0,0.0,0.0,202105,240.000000,0.0,0.0
347,d6fc923e-2ef2-4400-b5c5-5a59fd741346,0,0.0,0.0,202105,120.000000,0.0,1.0
363,89d27c28-077a-461a-bc48-797cafd73af9,0,0.0,0.0,202105,180.000000,0.0,1.0
...,...,...,...,...,...,...,...,...
236191,ecfc6bdc-c58b-4e54-b19a-2b4ce8df4340,0,0.0,0.0,202312,1620.000000,0.0,0.0
236193,b1ba4704-ec64-411c-b0ba-7453d11d4d56,0,0.0,0.0,202312,400.016949,0.0,1.0
236197,3c88fb21-2e3a-493f-a4dd-f747b4b30ee2,0,0.0,0.0,202312,2016.000000,0.0,1.0
236198,844e8dde-4b79-4a2e-a915-b7a1b5c2e817,0,0.0,0.0,202312,316.796610,0.0,1.0


In [53]:
fee_paid_data_  = fee_paid_data_[(fee_paid_data_['payment_max']>0)&(fee_paid_data_['user_id'].isin(users_4m.user_id))]
fee_paid_data_.groupby("new_policy_approved")["monthly_fee_paid"].sum().round(0)

# fee_paid_data_[["monthly_fee_paid","late_fee_paid"]].sum().round(0)

new_policy_approved
0.0     6457607.0
1.0    15054473.0
Name: monthly_fee_paid, dtype: float64

In [46]:
fee_paid_data_  = fee_paid_data_[(fee_paid_data_['payment_max']>0)&(fee_paid_data_['user_id'].isin(users_4m.user_id))]
fee_paid_data_.groupby("new_policy_approved")["late_fee_paid"].sum().round(0)

new_policy_approved
0.0    631424.0
1.0    885655.0
Name: late_fee_paid, dtype: float64

In [ ]:
(fee_paid_data_[(fee_paid_data_['DPD']==0) & (fee_paid_data_['payment_max']>0)&(fee_paid_data_['user_id'].isin(users_4m.user_id))]
    .groupby('billing_cycle')
    .agg(monthly_fee_paid = ('monthly_fee_paid','sum'),
    count_of_users = ('billing_cycle', 'count'),
    late_fee_paid = ("late_fee_paid", "sum"))).round(0)

# fee_paid_data_[(fee_paid_data_['DPD']==0) & (fee_paid_data_['payment_max']>0)&(fee_paid_data_[''].isin())]

,monthly_fee_paid,count_of_users,late_fee_paid
billing_cycle,,,
202104,7890.0,219,0.0
202105,39675.0,209,0.0
202106,38985.0,218,1500.0
202107,39543.0,215,2100.0
202108,43389.0,262,7920.0
202109,50314.0,312,5400.0
202110,57481.0,319,7650.0
202111,61754.0,453,15300.0
202112,86288.0,801,9600.0


In [75]:
4034*300

1210200

In [58]:
# final[~(final['DPD']>0)&(final['billing_cycle']=="202401")]

In [59]:
fee_paid_data

,user_id,billing_cycle,monthly_fee_paid,late_fee_paid,new_policy_approved
0,000ade4b-897e-450a-89f2-2f5e3078737e,202305,429.59322,0.0,0
1,000ade4b-897e-450a-89f2-2f5e3078737e,202306,429.59322,0.0,0
2,000ade4b-897e-450a-89f2-2f5e3078737e,202307,429.59322,0.0,0
3,000ade4b-897e-450a-89f2-2f5e3078737e,202308,429.59322,0.0,0
4,000ade4b-897e-450a-89f2-2f5e3078737e,202309,0.00000,0.0,0
...,...,...,...,...,...
128048,fd7ec252-76d6-43ac-8532-cf9bbe1ea24e,202208,0.00000,300.0,0
128049,ffe398bb-08eb-4570-be61-fa9792534075,202202,90.00000,0.0,1
128050,ffe398bb-08eb-4570-be61-fa9792534075,202205,90.00000,0.0,1
128051,ffe398bb-08eb-4570-be61-fa9792534075,202206,90.00000,0.0,1


In [60]:
user_level_data = final.groupby('user_id').agg(total_monthly_fee_paid = ('monthly_fee_paid', 'sum'),
                             total_late_fee_paid = ('late_fee_paid', 'sum'),
                             total_disbursal = ('total_disbursed', 'first'),
                             new_policy_approved = ('new_policy_approved', 'first'),
                             principal_outstanding = ('principal_outstanding', 'first'))

In [62]:
user_level_data.groupby("new_policy_approved")["total_late_fee_paid"].sum().round(0)

new_policy_approved
0     964188.0
1    1430889.0
Name: total_late_fee_paid, dtype: float64

In [63]:
dpd = final.drop_duplicates('user_id', keep= 'last')[['user_id', 'DPD']].reset_index(drop = True)

In [64]:
user_level_data = user_level_data.merge(dpd, on ='user_id', how = 'left')

In [65]:
user_level_data['total_disbursal'].sum().round(0)

518954598.0

In [66]:
user_level_data.shape

(11674, 7)

In [67]:
user_level_data['new_policy_approved'].sum().round(0)

8007

In [68]:
user_level_data['total_monthly_fee_paid'].sum().round(0)

30891536.0

In [69]:
user_level_data.shape

(11674, 7)

In [70]:
user_level_data.to_csv('user_level_data_portfolio_cutoff_aug24.csv', index = False)

In [71]:
user_level_data

,user_id,total_monthly_fee_paid,total_late_fee_paid,total_disbursal,new_policy_approved,principal_outstanding,DPD
0,000ade4b-897e-450a-89f2-2f5e3078737e,1718.372880,0.0,20395.0,0,6239.46,327.0
1,0018ec37-df63-4318-aaf6-eb27f851964b,9014.152543,0.0,209711.0,1,NaN,NaN
2,00283d2c-79cc-4e92-b915-47d224b01a10,1075.627119,0.0,6000.0,1,NaN,NaN
3,002a5e8d-aa19-4f27-81e4-d6c115ac26f6,138.000000,300.0,1900.0,1,NaN,NaN
4,002d305c-c671-4125-b8cb-ccd7b2a72563,1075.618644,0.0,9000.0,1,NaN,NaN
...,...,...,...,...,...,...,...
11669,ffe954a8-04dc-49ed-bdce-307956f59816,960.000000,0.0,36642.8,1,NaN,NaN
11670,fffa2453-fb42-4b5b-a68a-aeae8134e153,1557.050848,0.0,14405.0,1,NaN,NaN
11671,fffa3e6d-8f23-483e-9d1d-01fa70e08bab,1440.000000,300.0,29076.0,1,NaN,NaN
11672,fffe2a21-2db8-4402-b21e-4bf824900493,2249.500000,0.0,32101.0,1,NaN,NaN


In [72]:
user_level_data[user_level_data["DPD"]>=30]['principal_outstanding'].sum()

23178626.04

In [73]:
user_level_data['principal_outstanding'].sum()

23178628.27

### ninjakart

In [46]:
ninja = sanction_data[sanction_data['prod_group'] =='NINJA_1M']
ninja.shape

(180, 19)

In [47]:
ninjakart_disbursal = merchant_payments_data[merchant_payments_data['user_id'].isin(ninja.user_id)]


In [48]:
(ninjakart_disbursal.merge(billing_info[['user_id','billing_cycle', 'DPD','principal_outstanding']],
 on = ['user_id','billing_cycle'], how ='left'))

# ninjakart_disbursal.to_csv('ninjakart_disbursal_month_on_month.csv')

,user_id,billing_cycle,disbursal,DPD,principal_outstanding
0,017e3708-5ffe-4d2f-ab5b-686389548efa,202307,2703.0,0,0.0
1,017e3708-5ffe-4d2f-ab5b-686389548efa,202308,14891.0,0,891.0
2,033e538a-8752-4e7b-9c36-5ad2b6f29d64,202308,1000.0,0,0.0
3,033e538a-8752-4e7b-9c36-5ad2b6f29d64,202309,4000.0,0,0.0
4,072a17f7-f139-4dce-b561-480216de2414,202308,6000.0,0,6000.0
...,...,...,...,...,...
409,f910f72f-f40a-4de3-a9bb-2e6093d570cc,202307,4800.0,0,4800.0
410,f910f72f-f40a-4de3-a9bb-2e6093d570cc,202308,4800.0,0,4800.0
411,f9fda701-c80a-4c92-97e2-dea70998b699,202308,1000.0,0,1000.0
412,fb7a75a0-ad72-4600-8797-9d0b34f4b90e,202310,7200.0,0,7200.0


In [49]:
ninjakart_DPD = billing_info[(billing_info['billing_month']=='March')&
                   (billing_info['billing_year']=='2024')&
                   (billing_info['user_id'].isin(ninjakart_disbursal.user_id))][['user_id', 'DPD','principal_outstanding','payment_max','billing_cycle']]


In [50]:
fee_paid_data_ninjakart = pd.DataFrame({'user_id' : []})
billing_cycle_list = ['202403','202402','202401','202312','202311','202310','202309','202308','202307']
for i in range(len(billing_cycle_list)): 
    users_not_there = ninja[~(ninja['user_id'].isin(fee_paid_data_ninjakart.user_id))]
    print(len(users_not_there))
    fee_paid_chunk = statement_h_fun(users_not_there,billing_cycle_list[i])
    fee_paid_data_ninjakart = fee_paid_data_ninjakart.append(fee_paid_chunk)

180
163
161
158
158
158
158
158
158


In [51]:
tol_disbursed_amt = ninjakart_disbursal.groupby('user_id')['disbursal'].sum().rename('total_disbursed').reset_index()
final = tol_disbursed_amt.merge(ninjakart_DPD, on = 'user_id', how= 'left')
final = final.merge(fee_paid_data_ninjakart, on = 'user_id', how= 'left')



In [52]:
user_level_data_ninjakart = final.groupby('user_id').agg(total_monthly_fee_paid = ('monthly_fee_paid', 'sum'),
                             total_late_fee_paid = ('late_fee_paid', 'sum'),
                             total_disbursal = ('total_disbursed', 'first'),
                             principal_outstanding = ('principal_outstanding', 'first'),
                             DPD = ('DPD', 'first'))

In [53]:
user_level_data_ninjakart
user_level_data_ninjakart.to_csv('user_level_data_ninjakart_portfolio_cut.csv')

# Flipkart

In [54]:
flipkart = sanction_data[sanction_data['prod_group'] =='FLPKRT_1M']
flipkart.shape

(1018, 19)

In [55]:
merchant_payments = f"""
SELECT *, date_format(mp.created_time, '%Y%m') billing_cycle FROM merchant_payments AS mp
WHERE mp.`status` = 'success' and user_id in {tuple(flipkart.user_id)}
"""
parse_date = {}
# otp_varified_user=pd.read_sql(otp_varified, db_connection)
flag, merchant_payments_data_flipkart, error = utility.data_fetch_v2(merchant_payments, parse_dates=parse_date)
merchant_payments_data_flipkart.shape

(9319, 23)

In [56]:
merchant_payments_data_flipkart['beneficiary_account_name'].value_counts()

Flipkart Wholesale               1991
Walmart India Private Limited    1037
Premshankar Sahu                   64
Deepak Lodhi                       50
Yash Pet Store                     49
                                 ... 
Ramesh Chandra Samdani              1
Mr Sooraj Kumar Gupta               1
Chandra Prakash Singh               1
Chhagan Lal Mundra                  1
Coco Aliganj                        1
Name: beneficiary_account_name, Length: 2714, dtype: int64

In [57]:
merchant_payments_data_flipkart['fee'] = np.where(merchant_payments_data_flipkart['beneficiary_account_name'].isin(['Flipkart Wholesale','Walmart India Private Limited']),merchant_payments_data_flipkart['amount']*0.005, 0 )



In [58]:
merchant_payments_data_flipkart['fee'].sum()
# merchant_payments_data_flipkart.shape

107266.6023

In [59]:
merchant_payments_data_flipkart[['amount','fee']]


,amount,fee
0,10.00,0.0500
1,11988.72,59.9436
2,12000.00,60.0000
3,9172.00,45.8600
4,8765.00,43.8250
...,...,...
9314,8.00,0.0000
9315,4528.00,0.0000
9316,29000.00,0.0000
9317,29000.00,0.0000


In [195]:
flipkart_disbursal = merchant_payments_data[merchant_payments_data['user_id'].isin(flipkart.user_id)]

In [257]:
merchant_payments_data

,user_id,billing_cycle,disbursal
0,000ade4b-897e-450a-89f2-2f5e3078737e,202304,4832.0
1,000ade4b-897e-450a-89f2-2f5e3078737e,202305,4778.0
2,000ade4b-897e-450a-89f2-2f5e3078737e,202306,4799.0
3,000ade4b-897e-450a-89f2-2f5e3078737e,202307,4800.0
4,000ade4b-897e-450a-89f2-2f5e3078737e,202308,1186.0
...,...,...,...
93602,fffe2a21-2db8-4402-b21e-4bf824900493,202302,5000.0
93603,fffe2a21-2db8-4402-b21e-4bf824900493,202303,10001.0
93604,fffe2a21-2db8-4402-b21e-4bf824900493,202304,10000.0
93605,fffe2a21-2db8-4402-b21e-4bf824900493,202305,7000.0


In [187]:
flipkart_DPD = billing_info[(billing_info['billing_month']=='March')&
                   (billing_info['billing_year']=='2024')&
                   (billing_info['user_id'].isin(flipkart_disbursal.user_id))][['user_id', 'DPD','principal_outstanding','payment_max']]


In [188]:
fee_paid_data_flipkart = pd.DataFrame({'user_id' : []})
billing_cycle_list = ['202403','202402','202401','202312','202311','202310','202309','202308','202307', '202306','202305']
for i in range(len(billing_cycle_list)): 
    users_not_there = flipkart[~(flipkart['user_id'].isin(fee_paid_data_flipkart.user_id))]
    print(len(users_not_there))
    fee_paid_chunk = statement_h_fun(users_not_there,billing_cycle_list[i])
    fee_paid_data_flipkart = fee_paid_data_flipkart.append(fee_paid_chunk)

1018
1018
962
597
589
588
586
584
582
582
576


In [193]:
flipkart_disbursal.to_csv('flipkart_disbursal_month_om_month.csv')

In [189]:
tol_disbursed_amt = flipkart_disbursal.groupby('user_id')['disbursal'].sum().rename('total_disbursed').reset_index()
final = tol_disbursed_amt.merge(flipkart_DPD, on = 'user_id', how= 'left')
final = final.merge(fee_paid_data_flipkart, on = 'user_id', how= 'left')


In [190]:
user_level_data_flipkart = final.groupby('user_id').agg(total_monthly_fee_paid = ('monthly_fee_paid', 'sum'),
                             total_late_fee_paid = ('late_fee_paid', 'sum'),
                             total_disbursal = ('total_disbursed', 'first'),
                             principal_outstanding = ('principal_outstanding', 'first'),
                             DPD = ('DPD', 'first'))

In [191]:
user_level_data_flipkart.to_csv('user_level_data_flipkart_portfolio_cut.csv')